# Importing Tensorflow=1.13.2

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==1.13.2 
# MUST restart runtime after running this 

Uninstalling tensorflow-2.4.1:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.4.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
y
y
y
y
y
y
y
y
y
y
y
y
yy

  Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 92.7MB 1.3MB/s 
     |████████████████████████████████| 3.2MB 45.2MB/s 
     |████████████████████████████████| 368kB 44.0MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4

In [ ]:
# checking tensorflow version
import tensorflow as tf
from tensorflow.contrib.tensor_forest import tensor_forest
from tensorflow.python.ops import resources
print(tf.__version__)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

1.13.2


In [ ]:
# Import libraries
from __future__ import print_function
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import time 
# Ignore all GPUs, tf random forest does not benefit from it.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# normalize [-1,1]
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = 2*((df[feature_name] - min_value) / (max_value - min_value))-1
    return result

# normalize [0,1] - not using 
def normalize01(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = 2*((df[feature_name] - min_value) / (max_value - min_value))-1
    return result

In [ ]:
sheet_name = ['BSE_Sensex', 'NIFTY_50', 'Reliance', 'Infosys']
BSE_Sensex_feature = pd.read_excel("/content/gdrive/My Drive/MIE424-Project/data_feature_more.xlsx",sheet_name=sheet_name[0])
NIFTY_50_feature = pd.read_excel("/content/gdrive/My Drive/MIE424-Project/data_feature_more.xlsx",sheet_name=sheet_name[1])
Reliance_feature = pd.read_excel("/content/gdrive/My Drive/MIE424-Project/data_feature_more.xlsx",sheet_name=sheet_name[2])
Infosys_feature = pd.read_excel("/content/gdrive/My Drive/MIE424-Project/data_feature_more.xlsx",sheet_name=sheet_name[3])

# Random Forest Model 1 (~51% accuracy)

In [ ]:
def run_random_forest(data, num_steps, num_trees, max_nodes):
    data = data.dropna() # DROPNA
    tf.reset_default_graph()
    

    #Extract feature and target np arrays (inputs for placeholders)
    input_x = normalize(data.iloc[:, 1:-1]).values  # NORMALIZE 
    input_y = data.iloc[:, -1:].values

    ##### 10 input features
    # Splitting the dataset into the Training set and Test set
    # X_train, X_test, y_train, y_test = train_test_split(input_x, input_y, test_size = 0.25, random_state = 0)

    ##### time series features
    # Construct a time series input features
    # as defined by how many prior days of input features to be used to predict today's stock movement
    num_days = 3
    new_input_x = []
    for i in range (num_days, input_x.shape[0]+1):
        new_input_x.append(input_x[i-num_days: i, :].flatten())
    new_input_x = np.array(new_input_x)

    new_input_y = input_y[num_days-1:]

    # Splitting the dataset into the Training set and Test set
    X_train, X_test, y_train, y_test = train_test_split(new_input_x, new_input_y, test_size = 0.2, random_state = 424)


    # Fixed Parameters
    num_classes = 2 # up and down movement
    num_features = 10*num_days # 10 features

    # Input and Target placeholders 
    X = tf.placeholder(tf.float32, shape=[None, num_features])
    Y = tf.placeholder(tf.int64, shape=[None,1])

    # Random Forest Parameters
    hparams = tensor_forest.ForestHParams(num_classes=num_classes, 
                                        num_features=num_features, 
                                        num_trees=num_trees, 
                                        max_nodes=max_nodes).fill()

    # Build the Random Forest
    forest_graph = tensor_forest.RandomForestGraphs(hparams)

    # Get training graph and loss
    train_op = forest_graph.training_graph(X, Y)
    loss_op = forest_graph.training_loss(X, Y)

    # Measure the accuracy
    infer_op, _, _ = forest_graph.inference_graph(X)
    correct_prediction = tf.equal(tf.argmax(infer_op, 1), tf.cast(Y, tf.int64))
    accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # Initialize the variables (i.e. assign their default value) and forest resources
    init_vars = tf.group(tf.global_variables_initializer(), resources.initialize_resources(resources.shared_resources()))
        
    # Start TensorFlow session
    sess = tf.Session()

    # Run the initializer
    sess.run(init_vars)

    # Training
    for i in range(1, num_steps + 1):
        _, l = sess.run([train_op, loss_op], feed_dict={X: X_train, Y: y_train})
        if i % 50 == 0 or i == 1:
            acc = sess.run(accuracy_op, feed_dict={X: X_train, Y: y_train})
            print('Step %i, Loss: %f, Acc: %f' % (i, l, acc))

    # Test Model
    print("Test Accuracy:", sess.run(accuracy_op, feed_dict={X: X_test, Y: y_test}))

## Results

In [ ]:
%%time
data_sets = [BSE_Sensex_feature, NIFTY_50_feature, Reliance_feature, Infosys_feature] 
#data_sets_name = ["BSE_Sensex", "NIFTY_50", "Reliance", "Infosys"] 
data_sets_name = ["BSE_Sensex"] 
num_steps = 100
num_trees = 150
max_nodes = 500
for i in range(len(data_sets_name)):
    print(f"\nData:{data_sets_name[i]}")
    data = data_sets[i]
    if "Close" in data.columns:
        del data["Close"]
    run_random_forest(data, num_steps, num_trees, max_nodes)


Data:BSE_Sensex
INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'num_trees': 150, 'max_nodes': 500, 'bagging_fraction': 1.0, 'feature_bagging_fraction': 1.0, 'num_splits_to_consider': 10, 'max_fertile_nodes': 0, 'split_after_samples': 250, 'valid_leaf_threshold': 1, 'dominate_method': 'bootstrap', 'dominate_fraction': 0.99, 'model_name': 'all_dense', 'split_finish_name': 'basic', 'split_pruning_name': 'none', 'collate_examples': False, 'checkpoint_stats': False, 'use_running_stats_method': False, 'initialize_average_splits': False, 'inference_tree_paths': False, 'param_file': None, 'split_name': 'less_or_equal', 'early_finish_check_every_samples': 0, 'prune_every_samples': 0, 'num_classes': 2, 'num_features': 30, 'bagged_num_features': 30, 'bagged_features': None, 'regression': False, 'num_outputs': 1, 'num_output_columns': 3, 'base_random_seed': 0, 'leaf_model_type': 0, 'stats_model_type': 0, 'finish_type': 0, 'pruning_type': 0, 'split_type': 0}
Step 1, Loss: -1.0

## Tuning Hyperparameters for BSE_Sensex

In [ ]:
# Tuning Hyperparameters for BSE_Sensex

data = BSE_Sensex_feature 
data_name =  "BSE_Sensex" 
num_steps_tune = [200, 400, 600, 800, 1000, 1200, 1400] # Total steps to train
num_trees_tune = [10, 50, 100, 150, 200] 
max_nodes_tune = [100, 500, 1000] 

for num_steps in num_steps_tune:
    for batch_size in batch_size_tune:
        for num_trees in num_trees_tune:
            for max_nodes in max_nodes_tune: 
                print(f"data:{data_name}")
                print(f"num_steps:{num_steps}, num_trees:{num_trees}, max_nodes:{max_nodes}")
                run_random_forest(data, num_steps, num_trees, max_nodes)
                print()

data:BSE_Sensex
num_steps:200, batch_size:32, num_trees:10, max_nodes:100
INFO:tensorflow:Constructing forest with params = 
INFO:tensorflow:{'num_trees': 10, 'max_nodes': 100, 'bagging_fraction': 1.0, 'feature_bagging_fraction': 1.0, 'num_splits_to_consider': 10, 'max_fertile_nodes': 0, 'split_after_samples': 250, 'valid_leaf_threshold': 1, 'dominate_method': 'bootstrap', 'dominate_fraction': 0.99, 'model_name': 'all_dense', 'split_finish_name': 'basic', 'split_pruning_name': 'none', 'collate_examples': False, 'checkpoint_stats': False, 'use_running_stats_method': False, 'initialize_average_splits': False, 'inference_tree_paths': False, 'param_file': None, 'split_name': 'less_or_equal', 'early_finish_check_every_samples': 0, 'prune_every_samples': 0, 'num_classes': 2, 'num_features': 10, 'bagged_num_features': 10, 'bagged_features': None, 'regression': False, 'num_outputs': 1, 'num_output_columns': 3, 'base_random_seed': 0, 'leaf_model_type': 0, 'stats_model_type': 0, 'finish_type': 0

KeyboardInterrupt: ignored